In [ ]:
import copy

In [ ]:
def preprocess(fname):
    guard_map = []
    state = {}

    with open(fname, "r") as f:
        for li, line in enumerate(f.readlines()):
            guard_map.append(line.strip())
            if "^" in line:
                ci = line.find("^")
                state["pos"] = (li, ci)
                state["dir"] = "up"
                state["inside"] = True
    return guard_map, state

In [ ]:
next_dir = {"up": "right",
          "right": "down",
          "down": "left",
          "left": "up"}

def move(guard_map, state):
    move_dir = state["dir"]
    curr_row, curr_col = state["pos"]
    if move_dir == "up":
        new_row = curr_row - 1
        new_col = curr_col
        if new_row < 0:
            state["inside"] = False
    elif move_dir == "down":
        new_row = curr_row + 1
        new_col = curr_col 
        if new_row > len(guard_map) - 1:
            state["inside"] = False
    elif move_dir == "left":
        new_row = curr_row 
        new_col = curr_col - 1
        if new_col < 0:
            state["inside"] = False
    elif move_dir == "right":
        new_row = curr_row 
        new_col = curr_col + 1
        if new_col > len(guard_map[new_row]) - 1:
            state["inside"] = False

    if state["inside"] == False:
        return state
    
    # check obstacle
    if guard_map[new_row][new_col] == "#":
        state["pos"] = (curr_row, curr_col)
        state["dir"] = next_dir[state["dir"]]
        state = move(guard_map, state)
    else:
        state["pos"] = (new_row, new_col)
    return state

def part1(guard_map, state):
    visited = []
    visited.append((state["pos"], state["dir"]))
    distinct_pos = []
    while True:
        state = move(guard_map, state)
        if state["inside"] and (state["pos"], state["dir"]) not in visited:
            visited.append((state["pos"], state["dir"]))
        else:
            for pos, dir in visited:
                if pos not in distinct_pos:
                    distinct_pos.append(pos)
            return distinct_pos

guard_map, state = preprocess("day06_example.txt")

part1_example_sol = part1(guard_map, state)
print(f"Part 1 solution for example data: {len(part1_example_sol)}")
assert len(part1_example_sol) == 41

guard_map, state = preprocess("day06_input.txt")

part1_sol = part1(guard_map, state)
print(f"Part 1 solution: {len(part1_sol)}")

In [ ]:
def mod_map(guard_map, pos):
    row, col = pos
    line = list(guard_map[row])
    line[col] = "#"
    guard_map[row] = "".join(line)
    return guard_map

def check_loop(guard_map, state):
    visited = []
    visited.append((state["pos"], state["dir"]))
    while True:
        state = move(guard_map, state)

        if not state["inside"]:
            return False
        elif (state["pos"], state["dir"]) in visited:
            return True
        elif state["inside"]:
            visited.append((state["pos"], state["dir"]))
        
def part2(guard_map, state, distinct_pos):
    loops = 0
    loop_pos = []
    for pos in distinct_pos[1:]:
        new_guard_map = mod_map(copy.deepcopy(guard_map), pos)
        if check_loop(new_guard_map, copy.deepcopy(state)):
            loop_pos.append(pos)
            loops += 1
    return loops

guard_map, state = preprocess("day06_example.txt")

new_guard_map = mod_map(copy.deepcopy(guard_map), (6, 3))
assert check_loop(new_guard_map, copy.deepcopy(state)) == True

new_guard_map = mod_map(copy.deepcopy(guard_map), (7, 6))
assert check_loop(new_guard_map, copy.deepcopy(state)) == True

part2_example_sol = part2(guard_map, state, part1_example_sol)

print(f"Part 2 solution for example data: {part2_example_sol}")
assert part2_example_sol == 6

guard_map, state = preprocess("day06_input.txt")

part2_sol = part2(guard_map, state, part1_sol)
print(f"Part 2 solution: {part2_sol}")